In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Werner_Syndrome"
cohort = "GSE62877"

# Input paths
in_trait_dir = "../../input/GEO/Werner_Syndrome"
in_cohort_dir = "../../input/GEO/Werner_Syndrome/GSE62877"

# Output paths
out_data_file = "../../output/preprocess/Werner_Syndrome/GSE62877.csv"
out_gene_data_file = "../../output/preprocess/Werner_Syndrome/gene_data/GSE62877.csv"
out_clinical_data_file = "../../output/preprocess/Werner_Syndrome/clinical_data/GSE62877.csv"
json_path = "../../output/preprocess/Werner_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE62877-GPL14592_series_matrix.txt.gz', 'GSE62877-GPL5175_series_matrix.txt.gz', 'GSE62877_family.soft.gz']
Identified SOFT files: ['GSE62877_family.soft.gz']
Identified matrix files: ['GSE62877-GPL14592_series_matrix.txt.gz', 'GSE62877-GPL5175_series_matrix.txt.gz']

Background Information:
!Series_title	"Werner syndrome WRN helicase alters gene expression in a G-quadruplex DNA-dependent manner to antagonize a pro-senescence gene expression program"
!Series_summary	"Werner syndrome (WS) is a human adult progeroid syndrome caused by loss-of-function mutations in the WRN RECQ helicase gene. We analyzed mRNA and miRNA expression in fibroblasts from WS patients and in fibroblasts depleted of WRN protein in order to determine the role of WRN in transcription regulation, and to identify genes and miRNAs that might drive WS disease pathogenesis. Genes altered in WS cells participate in cellular growth, proliferation and survival; in tRNA charging and in onco

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import gzip
import pandas as pd
import numpy as np
from typing import Optional, Dict, Any, Callable, Union

# 1. Check Gene Expression Data Availability
# Given the background info, this dataset contains mRNA expression data
is_gene_available = True

# 2. Identify row indices for trait, age, and gender data in sample characteristics
# 2.1 Data availability

# For trait (Werner Syndrome): looking at group field in row 2
trait_row = 2  # Contains "group: WRN" vs "group: control" or "group: NS"

# For age: looking at row 4 which contains age information
age_row = 4  # Contains "age: X" values

# For gender: looking at row 5 which contains gender information
gender_row = 5  # Contains "gender: Female" or "gender: Male"

# 2.2 Define conversion functions for each variable

def convert_trait(value: str) -> int:
    """Convert trait value to binary (1 for Werner Syndrome, 0 for control)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # WRN = Werner Syndrome (1), Control/NS = not Werner Syndrome (0)
    if value.upper() == "WRN":
        return 1
    elif value.upper() in ["CONTROL", "NS"]:
        return 0
    else:
        return None

def convert_age(value: str) -> Union[float, None]:
    """Convert age value to continuous type"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        # Convert to float
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Union[int, None]:
    """Convert gender value to binary (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Female = 0, Male = 1
    if value.upper() in ["FEMALE", "F"]:
        return 0
    elif value.upper() in ["MALE", "M"]:
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
# Determine trait availability based on trait_row
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait_row is not None
if trait_row is not None:
    try:
        # Find all matrix files in the directory
        matrix_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('_series_matrix.txt.gz')]
        
        if matrix_files:
            # Use the first matrix file
            matrix_file_path = os.path.join(in_cohort_dir, matrix_files[0])
            
            # Read the file line by line to extract the sample characteristics
            clinical_data = {}
            row_index = 0
            with gzip.open(matrix_file_path, 'rt') as f:
                for line in f:
                    line = line.strip()
                    if line.startswith('!Sample_characteristics_ch1'):
                        # Extract the sample characteristics
                        parts = line.split('\t')
                        # The first element is the header, rest are values
                        values = parts[1:]
                        
                        # Add to our clinical data dictionary
                        clinical_data[row_index] = values
                        row_index += 1
            
            # Convert to DataFrame
            clinical_df = pd.DataFrame(clinical_data).T  # Transpose to get rows as in the dict
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_df, 
                trait=trait, 
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted features
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("No matrix files found in the directory.")
    
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If there's an error, at least let's try to save the validation
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


Preview of selected clinical features:
{0: [nan, nan, nan], 1: [nan, nan, nan], 2: [nan, nan, nan], 3: [nan, nan, nan], 4: [nan, nan, nan], 5: [nan, nan, nan], 6: [nan, nan, nan], 7: [nan, nan, nan], 8: [nan, nan, nan], 9: [nan, nan, nan], 10: [nan, nan, nan], 11: [nan, nan, nan], 12: [nan, nan, nan], 13: [nan, nan, nan], 14: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Werner_Syndrome/clinical_data/GSE62877.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data shape: (10240, 15)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs or other technical identifiers rather than
# standard human gene symbols. Human gene symbols typically follow a specific format
# (like BRCA1, TP53, etc.) with letters and sometimes numbers.
# These purely numeric IDs are likely platform-specific identifiers that need mapping.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
    GSM1317006  GSM1317010  GSM1317011  GSM1317013  GSM1317016
ID                                                            
1        25518       31913       19889       43757       17311
2        25467       29581       20771       42873       19908
3           26         215         334         195          86
4           94         239         318         175          79
5          389         128         273         243         215



Platform information:
!Series_title = Werner syndrome WRN helicase alters gene expression in a G-quadruplex DNA-dependent manner to antagonize a pro-senescence gene expression program
!Platform_title = [HuEx-1_0-st] Affymetrix Human Exon 1.0 ST Array [transcript (gene) version]
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = June 03, 2009: annotation table updated with netaffx build 28
!Platform_description = Oct 11, 2012: annotation table updated with netaffx build 32
#mrna_assignment = Description of the public mRNAs that should be detected by the sets within this transcript cluster based on sequence alignment
!Platform_title = OSU_CCC v4.0 [full array layout]
!Platform_title = nCounter Human miRNA Expression Assay, V2
!Sample_description = primary fibroblast cells
!Sample_description = primary fibroblast c


Gene annotation columns:
['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']

Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# Identify the correct columns for gene identifiers and gene symbols
# Based on the preview, we need to use 'ID' as the probe identifier and 'gene_assignment' for gene symbols
try:
    # Create the gene mapping dataframe
    mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
    
    # Preview the mapping to verify
    print("Preview of gene mapping (first 5 rows):")
    print(mapping_df.head())
    
    # Apply the gene mapping to convert probe-level measurements to gene expression data
    gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)
    
    # Check that the mapping was successful
    print(f"\nAfter mapping: Gene expression data shape: {gene_data.shape}")
    print("First 10 gene symbols:")
    print(gene_data.index[:10])
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene mapping: {e}")
    # If we failed to map, we should update our metadata
    is_gene_available = False
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Preview of gene mapping (first 5 rows):
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---

After mapping: Gene expression data shape: (0, 15)
First 10 gene symbols:
Index([], dtype='object', name='Gene')
Gene expression data saved to ../../output/preprocess/Werner_Syndrome/gene_data/GSE62877.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (0, 15)
First few gene symbols after normalization: []
Normalized gene data saved to ../../output/preprocess/Werner_Syndrome/gene_data/GSE62877.csv
Loaded clinical data:
      1   2   3   4   5   6   7   8   9  10  11  12  13  14
0                                                          
NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
Number of common samples between clinical and genetic data: 0
Clinical data index: [nan, nan, nan]
Gene data columns: ['GSM1317006', 'GSM1317010', 'GSM1317011', 'GSM1317013', 'GSM1317016', '...']
Extracted 15 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
            Werner_Syndrome
GSM1317006                1
GSM1317010                1
GSM1317011                1
GSM1317013                1
GSM1317016                1
Gene data shape for linking (samples as 